In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest", temperature=0.1, api_key="Your Google Api Key")

In [13]:
from langchain_community.tools import DuckDuckGoSearchRun # create tools

def search_duckduckgo(query: str): #web search tool
    """Searches DuckDuckGo using LangChain's DuckDuckGoSearchRun tool."""
    search = DuckDuckGoSearchRun()
    return search.invoke(query)

#Example usage
#result = search_duckduckgo("what are AI agent")
#print(result)

def add(a: float, b: float) -> float:
    """Add two numbers."""
    return a + b

def multiply(a: float, b: float) -> float:
    """Multiply two numbers."""
    return a * b

In [14]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

math_agent = create_react_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

In [15]:
research_agent = create_react_agent(
    model=model,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="You are a world class researcher with access to web search. Do not do any math."
)

In [16]:
# Create supervisor workflow
workflow = create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert. "
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [17]:
# Compile and run
app = workflow.compile()

result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is quantum computing?"
        }
    ]
})

In [18]:
result

{'messages': [HumanMessage(content='what is quantum computing?', additional_kwargs={}, response_metadata={}, id='cbd4ebc2-e966-46e1-ae7a-a87d0daf24c0'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'transfer_to_research_expert', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='supervisor', id='run-e49ee9c2-a926-4479-8967-16a5e6c28823-0', tool_calls=[{'name': 'transfer_to_research_expert', 'args': {}, 'id': 'b5efc939-433a-4889-b394-626cdef940b9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 7, 'total_tokens': 70, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Successfully transferred to research_expert', name='transfer_to_research_expert', id='47ae0dd1-6ce3-4f92-b4c4-5e23c55c400c', tool_call_id='b5efc939-433a-4889-b394-626cdef940b9'),
  AIMessage(content='Quantum computing harnesses the mind-bending principl

In [19]:
for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is quantum computing?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (b5efc939-433a-4889-b394-626cdef940b9)
 Call ID: b5efc939-433a-4889-b394-626cdef940b9
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

Quantum computing harnesses the mind-bending principles of quantum mechanics to perform powerful calculations. Unlike classical computers that store information as bits representing 0 or 1, quantum computers use qubits.  Qubits leverage superposition and entanglement, allowing them to represent 0, 1, or a combination of both simultaneously. This unlocks the potential to tackle complex probl

In [22]:
# Compile and run
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": "what is the weather in delhi today. Multiply it by 2 and add 5"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

what is the weather in delhi today. Multiply it by 2 and add 5
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_research_expert (b0dc2e57-8fb1-4049-8eb5-5791b6a50448)
 Call ID: b0dc2e57-8fb1-4049-8eb5-5791b6a50448
  Args:
================================= Tool Message =================================
Name: transfer_to_research_expert

Successfully transferred to research_expert
================================== Ai Message ==================================
Name: research_expert

I cannot answer your question because the available tools are not working as expected. Please try again later.
================================== Ai Message ==================================
Name: research_expert

Transferring back to supervisor
Tool Calls:
  transfer_back_to_supervisor (39d043e2-a51f-4259-97ec-c1e9b36c3397)
 Call ID: 39d043e2-a51f-4259-